In [14]:
# n = 104729 * 103591
# step = 25000
# B = 25000

In [15]:
n = 97 * 31
n = 389 * 401
n = 72869 * 401
B = 300
step = 100

In [16]:
import decimal
import numpy as np
from time import time
from QSlib import *
import pandas as pd

## Primes

In [22]:
class Primes:
    def __init__(self, n, B, q):
#         eratosthenes
        numbers = list(range(2, B + 1)) # changed from 2 to 3
        for number in numbers:
            if number != 0:
                for candidate in range(2 * number, B+1, number):
                    numbers[candidate-2] = 0
        primes = list(filter(lambda x: x != 0, numbers))
        
        self.p = []
        self.r = []
        q = Q(n)

#         for 2 only у него один корень
        if legendre(n % primes[0], primes[0]) == 1:
            tr = tonelli(n,primes[0])
            r = [(tr - q.m) % primes[0]]
            self.p.append(primes[0])
            self.r.append(r)

        # Для каждого прайма проверяем Лежандра и находим корни Тонелли Шенксом
        for i in range(1,len(primes)):
            if legendre(n % primes[i], primes[i]) == 1:
                tr = tonelli(n,primes[i])
                r = [(tr - q.m) % primes[i],(primes[i] - tr - q.m) % primes[i]]
                self.p.append(primes[i])
                self.r.append(r)

        print("primes len",len(self.p))
    def __getitem__(self,i):
        return self.p[i]
    def __len__(self):
        return len(self.p)

## Sieve

In [18]:
def suive(q, primes):
    print("step:",step)
    k = 1
    smooth_numbers = []
    while q((k-1)*step) < n:
        ans = smooth_region((k-1)*step,k*step,q,primes)
        for i in range(len(ans)):
            smooth_numbers.append(ans[i])
        ans = smooth_region(-k*step,-(k-1)*step,q,primes)
        for i in range(len(ans)):
            smooth_numbers.append(ans[i])
        k+=1
        print("Total number of smooth numbers:",len(smooth_numbers))
        if len(smooth_numbers) > len(primes):
            # Выброс из функции
            return smooth_numbers

## Matrix solver

In [19]:
class Matrix_solver:
    def __init__(self, primes, n):
        self.matrix = []
        self.primes = primes
        self.num_smooth_numbers = 0
        self.n = n
        self.m = int(decimal.Decimal(n).sqrt() + 1)

    def add(self, smooth_number):
        if(self.num_smooth_numbers == len(self.primes) + 2):
            return False
        self.matrix.append([smooth_number[0]] + [smooth_number[1]] + [list(smooth_number[2])] + 
                           [[self.num_smooth_numbers]] + [self.m + smooth_number[0]] + [list(smooth_number[2])] + [1])
        self.num_smooth_numbers += 1

    def display(self):
        df = pd.DataFrame(self.matrix, columns=['i','q()','razl','del','1','1','1'])
        print(df, df.shape)

    def solve(self, smooth_numbers):
        self.display()
        print('----')
        
        for line in self.matrix:
            for i, item in enumerate(line[2]):
                line[2][i] = line[2][i] % 2

        ans = [None,None]
        
        visited = []
        for j in range(len(self.matrix[0][2])):
            
            piv = -1
            for i in range(len(self.matrix)):
                if i not in visited and self.matrix[i][2][j] == 1:
                    visited.append(i)
                    piv = i
                    break
            if piv != -1:
                for i in range(len(self.matrix)):
                    if i != piv and self.matrix[i][2][j] == 1:
                        for k in range(0, len(self.matrix[0][2])):
                            self.matrix[i][2][k] = (self.matrix[i][2][k] + self.matrix[piv][2][k]) % 2
                            
                        for k in range(0, len(self.matrix[0][2])):
                            self.matrix[i][5][k] += self.matrix[piv][5][k]
                            
                        self.matrix[i][3].append(self.matrix[piv][3][0])
                        self.matrix[i][4] *= self.m + self.matrix[piv][0]
            else:
                pass
        
        
        for i in range(len(self.matrix)):
            if 1 not in self.matrix[i][2]:
                for j in range(len(self.primes)):
                    self.matrix[i][5][j] //= 2
                for j in range(len(self.primes)):
                    if self.matrix[i][5][j] != 0:
                        self.matrix[i][6] *= self.primes[j] ** self.matrix[i][5][j]

                right = self.matrix[i][6]
                left = self.matrix[i][4]

                gcd = min(GCD(abs(int(left+right)), n), GCD(abs(int(left-right)), n))
                if gcd > 1:
                    ans = [gcd, n//gcd]
                    break
                    
        self.display()
        return ans

## Factor

In [20]:
def Factor(n,B):
    q = Q(n)
    primes = Primes(n,B,q)
    smooth_numbers = suive(q,primes)
    matrix = Matrix_solver(primes.p, n)
    
    for smooth in smooth_numbers:
        matrix.add(smooth)
    pd.options.display.expand_frame_repr = False
    solve = matrix.solve(smooth_numbers)
    return solve

## index

In [21]:
Factor(n, B)

primes len 1585
step: 100
Total number of smooth numbers: 12
    i     q()                razl  del    1                   1  1
0   0     352  [5, 1, 0, 0, 0, 0]  [0]  735  [5, 1, 0, 0, 0, 0]  1
1  15   22627  [0, 3, 0, 1, 0, 0]  [1]  750  [0, 3, 0, 1, 0, 0]  1
2  16   24128  [6, 0, 1, 0, 0, 1]  [2]  751  [6, 0, 1, 0, 0, 1]  1
3  22   33176  [3, 1, 1, 0, 0, 1]  [3]  757  [3, 1, 1, 0, 0, 1]  1
4  48   73216  [9, 1, 1, 0, 0, 0]  [4]  783  [9, 1, 1, 0, 0, 0]  1
5  66  101728  [5, 1, 0, 2, 0, 0]  [5]  801  [5, 1, 0, 2, 0, 0]  1
6  74  114608  [4, 0, 1, 0, 1, 1]  [6]  809  [4, 0, 1, 0, 1, 1]  1
7 -62  -86944  [5, 1, 1, 0, 1, 0]  [7]  673  [5, 1, 1, 0, 1, 0]  1 (8, 7)
----
    i     q()                razl              del                1                    1       1
0   0     352  [1, 0, 0, 1, 0, 0]           [0, 1]           551250   [5, 4, 0, 1, 0, 0]       1
1  15   22627  [0, 1, 0, 1, 0, 0]              [1]              750   [0, 3, 0, 1, 0, 0]       1
2  16   24128  [0, 0, 1, 0, 0, 0]

[277, 1949]